In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""Graph utilities."""

import logging
import sys
from os import path
from time import time
from glob import glob
from six.moves import range, zip, zip_longest
from six import iterkeys
from collections import defaultdict, Iterable
from multiprocessing import cpu_count
import random
import collections
from random import shuffle
from itertools import product,permutations
from scipy.io import loadmat
from scipy.sparse import issparse

from concurrent.futures import ProcessPoolExecutor

from multiprocessing import Pool
from multiprocessing import cpu_count



logger = logging.getLogger("deepwalk")


__author__ = "Bryan Perozzi"
__email__ = "bperozzi@cs.stonybrook.edu"

LOGFORMAT = "%(asctime).19s %(levelname)s %(filename)s: %(lineno)s %(message)s"


class Node(object):
    def __init__(self, id, name, type='user'):
        self.id = str(id)
        self.neighbors = []
        self.name = name
        self.type = type
        self.rating = {}

class Movie(object):
    def __init__(self, name):
        self.name = name
        self.director = None
        self.actors = [] 
        self.genres = []

In [2]:
class Graph(defaultdict):
  """Efficient basic implementation of nx `Graph' â€“ Undirected graphs with self loops"""  
  def __init__(self):
    super(Graph, self).__init__(list)

  def nodes(self):
    return self.keys()

  def adjacency_iter(self):
    return self.iteritems()

  def subgraph(self, nodes={}):
    subgraph = Graph()
    
    for n in nodes:
      if n in self:
        subgraph[n] = [x for x in self[n] if x in nodes]
        
    return subgraph

  def make_undirected(self):
  
    t0 = time()

    for v in self.keys():
      for other in self[v]:
        if v != other:
          self[other].append(v)
    
    t1 = time()
    logger.info('make_directed: added missing edges {}s'.format(t1-t0))

    self.make_consistent()
    return self

  def make_consistent(self):
    t0 = time()
    for k in iterkeys(self):
      self[k] = list(sorted(set(self[k])))
    
    t1 = time()
    logger.info('make_consistent: made consistent in {}s'.format(t1-t0))

    self.remove_self_loops()

    return self

  def remove_self_loops(self):

    removed = 0
    t0 = time()

    for x in self:
      if x in self[x]: 
        self[x].remove(x)
        removed += 1
    
    t1 = time()

    logger.info('remove_self_loops: removed {} loops in {}s'.format(removed, (t1-t0)))
    return self

  def check_self_loops(self):
    for x in self:
      for y in self[x]:
        if x == y:
          return True
    
    return False

  def has_edge(self, v1, v2):
    if v2 in self[v1] or v1 in self[v2]:
      return True
    return False

  def degree(self, nodes=None):
    if isinstance(nodes, Iterable):
      return {v:len(self[v]) for v in nodes}
    else:
      return len(self[nodes])

  def order(self):
    "Returns the number of nodes in the graph"
    return len(self)    

  def number_of_edges(self):
    "Returns the number of nodes in the graph"
    return sum([self.degree(x) for x in self.keys()])/2

  def number_of_nodes(self):
    "Returns the number of nodes in the graph"
    return order()

  def random_walk(self, path_length, alpha=0, rand=random.Random(), start=None):
    """ Returns a truncated random walk.

        path_length: Length of the random walk.
        alpha: probability of restarts.
        start: the start node of the random walk.
    """
    G = self
    if start:
      path = [start]
    else:
      # Sampling is uniform w.r.t V, and not w.r.t E
      path = [rand.choice(G.keys())]

    while len(path) < path_length:
      cur = path[-1]
      if len(G[cur]) > 0:
        if rand.random() >= alpha:
          path.append(rand.choice(G[cur]))
        else:
          path.append(path[0])
      else:
        break
    return path

# TODO add build_walks in here

def build_deepwalk_corpus(G, num_paths, path_length, alpha=0,
                      rand=random.Random(0)):
  walks = []

  nodes = list(G.nodes())
  
  for cnt in range(num_paths):
    rand.shuffle(nodes)
    for node in nodes:
      walks.append(G.random_walk(path_length, rand=rand, alpha=alpha, start=node))
  
  return walks

def build_deepwalk_corpus_iter(G, num_paths, path_length, alpha=0,
                      rand=random.Random(0)):
  walks = []

  nodes = list(G.nodes())

  for cnt in range(num_paths):
    rand.shuffle(nodes)
    for node in nodes:
      yield G.random_walk(path_length, rand=rand, alpha=alpha, start=node)


def clique(size):
    return from_adjlist(permutations(range(1,size+1)))


# http://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks-in-python
def grouper(n, iterable, padvalue=None):
    "grouper(3, 'abcdefg', 'x') --> ('a','b','c'), ('d','e','f'), ('g','x','x')"
    return zip_longest(*[iter(iterable)]*n, fillvalue=padvalue)

def parse_adjacencylist(f):
  adjlist = []
  for l in f:
    if l and l[0] != "#":
      introw = [int(x) for x in l.strip().split()]
      row = [introw[0]]
      row.extend(set(sorted(introw[1:])))
      adjlist.extend([row])
  
  return adjlist

def parse_adjacencylist_unchecked(f):
  adjlist = []
  for l in f:
    if l and l[0] != "#":
      adjlist.extend([[int(x) for x in l.strip().split()]])
  
  return adjlist

def load_adjacencylist(file_, undirected=False, chunksize=10000, unchecked=True):

  if unchecked:
    parse_func = parse_adjacencylist_unchecked
    convert_func = from_adjlist_unchecked
  else:
    parse_func = parse_adjacencylist
    convert_func = from_adjlist

  adjlist = []

  t0 = time()

  with open(file_) as f:
    with ProcessPoolExecutor(max_workers=cpu_count()) as executor:
      total = 0 
      for idx, adj_chunk in enumerate(executor.map(parse_func, grouper(int(chunksize), f))):
          adjlist.extend(adj_chunk)
          total += len(adj_chunk)
  
  t1 = time()

  logger.info('Parsed {} edges with {} chunks in {}s'.format(total, idx, t1-t0))

  t0 = time()
  G = convert_func(adjlist)
  t1 = time()

  logger.info('Converted edges to graph in {}s'.format(t1-t0))

  if undirected:
    t0 = time()
    G = G.make_undirected()
    t1 = time()
    logger.info('Made graph undirected in {}s'.format(t1-t0))

  return G 


def load_edgelist(file_, undirected=True):
  G = Graph()
  with open(file_) as f:
    for l in f:
      x, y = l.strip().split()[:2]
      x = int(x)
      y = int(y)
      G[x].append(y)
      if undirected:
        G[y].append(x)
  
  G.make_consistent()
  return G


def load_matfile(file_, variable_name="network", undirected=True):
  mat_varables = loadmat(file_)
  mat_matrix = mat_varables[variable_name]

  return from_numpy(mat_matrix, undirected)


def from_networkx(G_input, undirected=True):
    G = Graph()

    for idx, x in enumerate(G_input.nodes_iter()):
        for y in iterkeys(G_input[x]):
            G[x].append(y)

    if undirected:
        G.make_undirected()

    return G


def from_numpy(x, undirected=True):
    G = Graph()

    if issparse(x):
        cx = x.tocoo()
        for i,j,v in zip(cx.row, cx.col, cx.data):
            G[i].append(j)
    else:
      raise Exception("Dense matrices not yet supported.")

    if undirected:
        G.make_undirected()

    G.make_consistent()
    return G


def from_adjlist(adjlist):
    G = Graph()
    
    for row in adjlist:
        node = row[0]
        neighbors = row[1:]
        G[node] = list(sorted(set(neighbors)))

    return G


def from_adjlist_unchecked(adjlist):
    G = Graph()
    
    for row in adjlist:
        node = str(row[0])
        neighbors = map(str, row[1:])
        G[node] = neighbors

    return G

In [3]:
def load_movie_data():
    # Movie data files used for building the graph
    movies_directors_filename = "data/movie_directors.dat"
    movies_actors_filename = "data/movie_actors.dat"
    movies_genres_filename = "data/movie_genres.dat"
    movies_filename = "data/movies.dat"
    
    # Load the data about the movies into a dictionary
    # The dictionary maps a movie ID to a movie object
    # Also store the unique directors, actors, and genres
    movies = {}
    with open(movies_filename, "r") as fin:
        fin.next()  # burn metadata line
        for line in fin:
            m_id, name = line.strip().split()[:2]
            movies["m"+m_id] = Movie(name)
    
    directors = set([])
    with open(movies_directors_filename, "r") as fin:
        fin.next()  # burn metadata line
        for line in fin:
            m_id, director = line.strip().split()[:2]
            if "m"+m_id in movies:
                movies["m"+m_id].director = director
            directors.add(director)
    
    actors = set([])
    with open(movies_actors_filename, "r") as fin:
        fin.next()  # burn metadata line
        for line in fin:
            m_id, actor = line.strip().split()[:2]
            if "m"+m_id in movies:
                movies["m"+m_id].actors.append(actor)
            actors.add(actor)
    
    genres = set([])
    with open(movies_genres_filename, "r") as fin:
        fin.next()  # burn metadata line
        for line in fin:
            m_id, genre = line.strip().split()
            if "m"+m_id in movies:
                movies["m"+m_id].genres.append(genre)
            genres.add(genre)

    return movies, directors, actors, genres

In [4]:
adjlist_file = open("out.adj", 'w')
node_list_file = open("nodelist.txt", 'w')

In [5]:
# Load all the ratings for every user into a dictionary
    # The dictionary maps a user to a list of (movie, rating) pairs
    #   e.g., ratings[75] = [(3,1), (32,4.5), ...]
num_ratings = 0
ratings = collections.defaultdict(dict)
with open("data/train_user_ratings.dat", "r") as fin:
    fin.next()  # burn metadata line
    for line in fin:
        ls = line.strip().split("\t")
        user, movie, rating = ls[:3]
        rating = str(int(round(float(rating))))
        ratings["u"+user]["m"+movie] = rating
        num_ratings += 1

In [6]:
movies, directors, actors, genres = load_movie_data()

In [7]:
print ratings['u383']

{'m296': '4', 'm1961': '4', 'm292': '3', 'm357': '3', 'm161': '4', 'm480': '3', 'm165': '1', 'm350': '1', 'm50': '5', 'm208': '3', 'm4973': '5', 'm7063': '5', 'm589': '1', 'm586': '1', 'm367': '1', 'm26828': '1', 'm608': '4', 'm1393': '3', 'm457': '5', 'm593': '4', 'm27741': '5', 'm597': '1', 'm33794': '3', 'm33493': '4', 'm1704': '4', 'm1265': '4', 'm5989': '4', 'm1784': '4', 'm7361': '1', 'm32': '2', 'm1517': '1', 'm442': '1', 'm6377': '4', 'm1307': '4', 'm7023': '5', 'm1580': '3', 'm788': '1', 'm5445': '3', 'm786': '1', 'm45720': '4', 'm260': '4', 'm2997': '4', 'm380': '1', 'm3793': '1', 'm4886': '3', 'm1279': '5', 'm21': '4', 'm41585': '1', 'm3996': '4', 'm541': '5', 'm7153': '5', 'm1073': '3', 'm141': '5', 'm2858': '4', 'm1079': '5', 'm5618': '5', 'm39183': '5', 'm2762': '5', 'm1923': '3', 'm6711': '5', 'm924': '5', 'm6': '3', 'm1208': '5', 'm2683': '1', 'm648': '3', 'm1610': '4', 'm2324': '5', 'm1617': '4', 'm1527': '4', 'm4226': '5', 'm2571': '4', 'm27255': '5', 'm1196': '4', 'm

In [8]:
print movies['m4511']

In [9]:
nodelist = []
nodedict = {}

In [10]:
count = 0 
print ratings.keys()

['u383', 'u190', 'u533', 'u170', 'u894', 'u732', 'u127', 'u267', 'u1164', 'u1035', 'u636', 'u637', 'u1017', 'u937', 'u1122', 'u1160', 'u78', 'u175', 'u728', 'u75', 'u476', 'u477', 'u498', 'u493', 'u548', 'u1174', 'u1198', 'u580', 'u1047', 'u622', 'u545', 'u1117', 'u325', 'u922', 'u783', 'u788', 'u1118', 'u1224']


In [15]:
for user in ratings.keys():
        nodedict[user] = Node(count, user, 'user')
        nodelist.append(nodedict[user])
        count = count + 1

for movie in movies.keys():
    nodedict[movie] = Node(count, movie, 'movie')
    nodelist.append(nodedict[movie])
    count = count + 1

for director in directors:
    nodedict[director] = Node(count, director, 'director')
    nodelist.append(nodedict[director])
    count = count + 1

for genre in genres:
    nodedict[genre] = Node(count, genre, 'genre')
    nodelist.append(nodedict[genre])
    count = count + 1

for actor in actors:
    nodedict[actor] = Node(count, actor, 'actor')
    nodelist.append(nodedict[actor])
    count = count + 1

for movie in movies.keys():
    for i in range(1,6):
        nodedict[movie+"_"+str(i)] = Node(count, movie+"_"+str(i),'rating')
        nodelist.append(nodedict[movie+"_"+str(i)])
        count = count + 1

# Add edges between users and movie-rating nodes
# Add edges between movie ratings and movies
for user in ratings.keys():
    for movie, rating in ratings[user].items():
        nodedict[user].neighbors.append(nodedict[movie+"_"+rating])
        nodedict[movie+"_"+rating].neighbors.append(nodedict[user])
        nodedict[movie].neighbors.append(nodedict[movie+"_"+rating])
        nodedict[movie+"_"+rating].neighbors.append(nodedict[movie])
# Add edges between movies and directors
# Add edges between movies and actors
# Add edges between movies and genres
for movie, m in movies.items():
    if m.director is not None:
        nodedict[movie].neighbors.append(nodedict[m.director])
        nodedict[m.director].neighbors.append(nodedict[movie])
    if not m.actors:
        for actor in m.actors:
            nodedict[movie].neighbors.append(nodedict[actor])
            nodedict[actor].neighbors.append(nodedict[movie])
    if not m.genres:
        for genre in m.genres:
            nodedict[movie].neighbors.append(nodedict[genre])
            nodedict[genre].neighbors.append(nodedict[movie])

In [47]:
for user in ratings.keys():
    nodelist.append(Node(count, user, 'user'))
    nodedict[user] = count
    count = count + 1

In [48]:
for movie in movies.keys():
    nodelist.append(Node(count, movie, 'movie'))
    nodedict[movie] = count
    count = count + 1

In [49]:
for director in directors:
    nodelist.append(Node(count, director, 'director'))
    nodedict[director] = count
    count = count + 1

In [50]:
for genre in genres:
    nodelist.append(Node(count, genre, 'genre'))
    nodedict[genre] = count
    count = count + 1

In [51]:
for actor in actors:
    nodelist.append(Node(count, actor, 'actor'))
    nodedict[actor] = count
    count = count + 1

In [52]:
for movie in movies.keys():
    for i in range(1,6):
        nodelist.append(Node(count, movie+"_"+str(i),'rating'))
        nodedict[movie+"_"+str(i)] = count
        count = count + 1

In [53]:
for user in ratings.keys():
    for movie, rating in ratings[user].items():
        nodelist[nodedict[user]].neighbors.append(nodelist[nodedict[movie+"_"+rating]])
        nodelist[nodedict[movie+"_"+rating]].neighbors.append(nodelist[nodedict[user]])
        nodelist[nodedict[movie]].neighbors.append(nodelist[nodedict[movie+"_"+rating]])
        nodelist[nodedict[movie+"_"+rating]].neighbors.append(nodelist[nodedict[movie]])

In [54]:
for movie, m in movies.items():
    if m.director is not None:
        nodelist[nodedict[movie]].neighbors.append(nodelist[nodedict[m.director]])
        nodelist[nodedict[m.director]].neighbors.append(nodelist[nodedict[movie]])
    if not m.actors:
        for actor in m.actors:
            nodelist[nodedict[movie]].neighbors.append(nodelist[nodedict[actor]])
            nodelist[nodedict[actor]].neighbors.append(nodelist[nodedict[movie]])
    if not m.genres:
        for genre in m.genres:
            nodelist[nodedict[movie]].neighbors.append(nodelist[nodedict[genre]])
            nodelist[nodedict[genre]].neighbors.append(nodelist[nodedict[movie]])

In [17]:
for node in nodelist:
    node_list_file.write("%s\t%s\t%s\n" % (node.id, node.name, node.type))
    adjlist_file.write("%s " % node.id)
    for n in node.neighbors:
        adjlist_file.write("%s " % n.id)
    adjlist_file.write("\n")
adjlist_file.close()
node_list_file.close()